In [ ]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# Introduction to Tensor Flow

In this notebook we show how to find the minimum of the function
$$ x \mapsto \exp(x) - 2 \cdot x^2 + 1 $$
using the <a href="https://en.wikipedia.org/wiki/TensorFlow">TensorFlow</a> library. 
We plot this function using `numpy`, `matplotlib`, and `seaborn`.

In [ ]:
import numpy             as np
import matplotlib.pyplot as plt
import seaborn           as sns

First, we define the function $x \mapsto \exp(x) - 2 \cdot x^2 + 1$ as a Python function that can take a 
`numpy` array as its argument.

In [ ]:
def fm(x):
    return np.exp(x) - 2 * x**2 + 1

Next, we plot this function for all $x$ such that $-1 \leq x \leq 3$.

In [ ]:
Xs = np.arange(-1.0, 3, 0.01)
Ys = fm(Xs)
plt.figure(figsize=(12,12))
sns.set(style='whitegrid')
sns.lineplot(x=Xs, y=Ys, color='b')
plt.axvline(x=0.0, c='k')
plt.axhline(y=0.0, c='k')
plt.ylim(-0.5, 3.0)
plt.xlim(-1.0, 3,0)
plt.xlabel('x')
plt.ylabel('y')
plt.title('x |-> exp(x) - 2 * x**2 + 1')

For $x \geq 0$, the function $f$ seems to have a minimum somewhere between $2.0$ and $2.5$.  We want to compute this minimum numerically using <font style="color:blue;">gradient descent</font> via 
<a href="https://www.tensorflow.org">TensorFlow</a>, but we do not want to compute the gradient of 
$f$ ourselves.  In order to install `tensorflow`, the following command can be used:
```
    conda install -c conda-forge tensorflow
```

In [ ]:
import tensorflow as tf

We start by defining a variable $x$.  Later, we will define the function 
$$ f(x) := \exp(x) - 2 \cdot x^2 + 1 $$
and compute the value $x_0$ such that $f(x_0) \leq f(x)$ for all $x \geq 0$.  The variable $x$ is a single precision variable, hence we use <tt>tf.float32</tt> as its data type.  The variable is initialized to the value $1$.  We also assign a *name* to it, but this name is completely optional, since this name is only used when we 
print the variable.  Hence it is only useful for debugging.

In [ ]:
x = tf.Variable(1)
x

Since this is a variable that contains only a single number and not an array or a matrix, its shape is 
<tt>()</tt>.  The string `'var_x:0'` is an internal name used by TensorFlow to manage this variable. 
Note that TensorFlow has appended the string <tt>':0'</tt> at the end of the string `var_x` in order to ensure that this name is unique.

Let us define a <font style="color:blue;">cost function</font> $f$ using `tensorflow` next.  Mathematically, 
this cost function is the function $f$ from above:
$$ f(x) = \exp(x) - 2 \cdot x^2 + 1 $$
Note that we have used the variable `x` defined above in the right hand side of this definition.

In [ ]:
def f_and_fs(t):
    x = tf.Variable(t)
    with tf.GradientTape() as tape:
        y = tf.exp(x) - 2 * x * x + 1
    ys = tape.gradient(y, x) 
    return y.numpy(), ys.numpy()

In [ ]:
y, ys = f_and_fs(4.0)
y, ys

In [ ]:
def findMinimum(start, eps):
    t     = start
    f, fs = f_and_fs(t)
    print(f'cnt = 0, f({t}) = {f}, fs = {fs}')
    alpha = 0.1   # learning rate
    cnt   = 0     # number of iterations
    while True:
        cnt += 1
        tOld, fOld = t, f
        t    -= alpha * fs
        f, fs = f_and_fs(t)
        print(f'cnt = {cnt}, f({t}) = {f}, fs = {fs}')
        if abs(t - tOld) <= abs(t) * eps:
            return t, f, fs, cnt            
        if f >= fOld:     # f didn't decrease, learning rate is too high
            alpha *= 0.5  # decrease the learning rate
            print(f'decrementing: alpha = {alpha}')
            t, f = tOld, fOld    # reset t
            continue
        else:             # f has decreased
            alpha *= 1.2  # increase the learning rate
            print(f'incrementing: alpha = {alpha}')

In [ ]:
findMinimum(1.0, 1e-15)

Note that although we have used <font style="color:blue;">gradient descent</font>, we never had to calculate the 
<font style="color:blue;">derivative</font> of the function $f$.  This derivative has been calculated by TensorFlow instead. 